# Jenssen Shannon Distance using Tensorflow Probability

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions


In [ ]:
%%script echo skipping...

# KL model
def nll(X, dist):
  return - tf.reduce_mean(dist.log_prob(X))

@tf.function
def get_loss_and_grads(X_train, dist):
  with tf.GradientTape() as tape:
    tape.watch(dist.trainable_variables)
    loss = nll(X_train, dist)
  grads = tape.gradient(loss, dist.trainable_variables)
  return loss, grads

def fit_multivariate_normal(data, n_epochs=10, batch_size=100):
  dist = tfd.MultivariateNormalDiag(
    loc=tf.Variable(data.mean(axis=0), name='loc'),
    scale_diag=tf.Variable(np.ones(data.shape[1]), name='scale_diag'))

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)

  for _ in range(n_epochs):
    # for batch in np.array_split(data, 1 + (data.shape[0] // batch_size)):
    batch = data  # use all the data in each epoch
    loss, grads = get_loss_and_grads(batch, dist)
    optimizer.apply_gradients(zip(grads, dist.trainable_variables))
    # loc_value = dist.loc.value()
  return dist


In [ ]:

%%script echo skipping...

# node2node js distance matrix
n2n_jsd = np.zeros((len(node_dists), len(node_dists)))
node_means = {}
node_stds = {}

for i,lbl_i in tqdm(enumerate(node_dists.index)):
  for j, lbl_j in enumerate(node_dists.index):
    P = node_dists.loc[lbl_i]
    Q = node_dists.loc[lbl_j]
    M = tfd.MultivariateNormalDiag(
      loc=(P.mean() + Q.mean())/2.,
      scale_diag=(P.stddev() + Q.stddev())/2.
    )
    node_means[lbl_i] = P.mean().numpy()
    node_stds[lbl_i] = P.stddev().numpy()
    jsd = .5 * (P.kl_divergence(M) + Q.kl_divergence(M)).numpy()
    # print(X_dists.loc[lbl_i].trainable_variables[1].shape)
    n2n_jsd[i,j] = jsd

n2n_jsd = pd.DataFrame(n2n_jsd, index=node_dists.index.to_list(), columns=node_dists.index.to_list())
# node_dists = node_features.progress_apply(fit_multivariate_normal)